In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [4]:
os.getcwd()
testdf = pd.read_csv("test.csv",encoding="big5")
traindf = pd.read_csv("train.csv",encoding="Big5")

'/home/jovyan/work'

In [6]:
car_traindf=pd.read_csv('data_car_clients.csv',encoding="utf-8")

In [8]:
car_traindf

,subject,content
0,價格,因為森林人即將換代，這套系統沒必要裝在一款即將換代的車型上，因為肯定會影響價格。
1,價格,四驅價格貌似挺高的，高的可以看齊XC60了，看實車前臉有點違和感。不過大眾的車應該不會差。
2,價格,斯柯達要說質量，似乎比大眾要好一點，價格也低一些，用料完全一樣。我聽說過野帝，但沒聽說過你說...
3,價格,這玩意都是給有錢任性又不懂車的土豪用的，這價格換一次我妹夫EP020可以換三鍋了
4,價格,17價格忒高，估計也就是14-15左右。
5,價格,我開始就是榮放2.5 森林人2.5二選一 榮放主要是底盤質感不行 太硬 其次是...
6,價格,唉，這貨的價格死硬死硬的，低配版優惠1萬據說已經罕有了。
7,價格,價格的話只能說一般般吧，太倉前段時間定的比你便宜！
8,價格,聽過，價格太貴，但一直念念不忘
9,價格,恭喜恭喜，這個優惠不錯哦！


In [9]:
car_traindf["subject"].unique()

array(['價格', '配置', '操控', '舒適性', '油耗', '動力', '內飾', '安全性', '空間', '外觀'],
      dtype=object)

In [10]:
car_traindf["subject"].value_counts()

動力     2732
價格     1273
油耗     1082
操控     1036
舒適性     931
配置      853
安全性     573
內飾      536
外觀      489
空間      442
Name: subject, dtype: int64

In [27]:
new_train=car_traindf[(car_traindf["subject"]=='動力') | (car_traindf["subject"]=='配置') |(car_traindf["subject"]=='油耗')|\
                      (car_traindf["subject"]=='操控')| (car_traindf["subject"]=='舒適性')].reset_index(drop=True)

In [35]:

import jieba
def cutpoem(poem):
    return " ".join(jieba.cut(poem)).replace("\r", "").replace("\n", "")
new_train["content"] = new_train["content"].apply(cutpoem)
# new_train

In [23]:
!pip install jieba

     |████████████████████████████████| 7.3MB 716kB/s eta 0:00:01
  Created wheel for jieba: filename=jieba-0.39-cp37-none-any.whl size=7282593 sha256=ebec9dda4dc102099e4f85095cdee5d9969f42f39cb3109ee8972d40bef8dd71
  Stored in directory: /home/jovyan/.cache/pip/wheels/c9/c7/63/a9ec0322ccc7c365fd51e475942a82395807186e94f0522243
Successfully built jieba


In [36]:
transdict = {"動力":0, "配置":1, "油耗":2,"操控":3,"舒適性":4}
ytrain = new_train["subject"].replace(transdict)
# ytest = testdf["作者"].replace(transdict)

0       1
1       1
2       1
3       1
4       1
       ..
6629    3
6630    2
6631    2
6632    1
6633    0
Name: subject, Length: 6634, dtype: int64

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
trainvec = vec.fit_transform(new_train["content"])
testvec = vec.transform(new_train["content"])

In [39]:
len(vec.vocabulary_)

16772

In [40]:
print(vec.get_feature_names())

['000', '01', '020', '03', '030', '04', '040', '05', '06', '07', '08', '09', '0a78d51926ec16809infoc', '0l', '0t', '0w', '0w20', '0w21', '0w2o', '0w30', '0w40', '10', '100', '1000', '10000', '100km', '1010', '1024x600', '103', '104', '105000', '107', '108', '109', '10km20', '10w', '10x', '11', '110', '1100', '110km', '1111', '115', '1150', '117', '118', '11ej25', '11l', '11x', '12', '120', '1200', '12000', '120km', '121', '1213', '125', '126', '128', '129', '12l', '12s', '12v', '13', '130', '1300', '13000', '1300km', '130km', '135', '136', '138', '13800', '13l', '13xt', '14', '140', '1400', '145', '145000', '148', '14l', '14v', '15', '150', '1500', '15000', '150000', '15000km', '150p', '152', '15208aa160', '1528785224421', '1580', '1582866622645548980', '15cm', '15kg', '15w', '16', '160', '1600', '16000', '1600km', '164', '1680', '16g', '17', '170', '1700', '17000', '17045', '171', '171p', '175', '1750', '17w', '18', '180', '1800', '18000', '180mm', '184', '185', '186p', '188km', '18w'

In [41]:
xtrain,xtest,y1,y2=train_test_split(trainvec,ytrain, test_size=0.1)

In [42]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(xtrain, y1)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [43]:
pre = clf.predict(xtest)
from sklearn.metrics import accuracy_score
accuracy_score(pre, y2)

0.5210843373493976

In [45]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y2, pre))

,0,1,2,3,4
0,149,19,31,29,21
1,16,62,3,13,7
2,27,1,65,10,7
3,33,11,6,40,16
4,23,10,17,18,30


<664x16772 sparse matrix of type '<class 'numpy.int64'>'
	with 9319 stored elements in Compressed Sparse Row format>

In [47]:
xtrain,xtest,y1,y2=train_test_split(trainvec,ytrain, test_size=0.1)

In [55]:
clf.fit(xtrain, y1)
pre1 = clf.predict(xtest)
accuracy_score(pre1, y2)

0.8211678832116789

In [56]:
pd.DataFrame(confusion_matrix(y2, pre1))

,0,1,2
0,79,7,9
1,17,91,9
2,5,2,55
